# Analysis of Normalized Queries

This notebook contains an analysis on CIViC evidence data

In [1]:
from pathlib import Path 
from enum import Enum
import zipfile 

import pandas as pd
from civicpy import civic as civicpy
import plotly.express as px

In [2]:
# Use latest cache that has been pushed to the repo
latest_cache_zip_path = sorted(Path().glob("cache-*.pkl.zip"))[-1]
print(f"Using {latest_cache_zip_path} for civicpy cache")

with zipfile.ZipFile(latest_cache_zip_path, "r") as zip_ref:
    zip_ref.extractall()

civicpy.load_cache(local_cache_path=Path("cache.pkl"), on_stale="ignore")

Using cache-20230531.pkl.zip for civicpy cache


True

## Total Variants in CIViC

In [3]:
civic_variant_ids = civicpy.get_all_variants()
total_number_variants = len(civic_variant_ids)
f"Total Number of variants in CIViC: {total_number_variants}"

'Total Number of variants in CIViC: 3554'

## Total Evidence items in CIViC

Need to remove all rejected evidence items

In [4]:
civic_evidence_ids = civicpy.get_all_evidence(include_status=["accepted", "submitted"])

In [5]:
total_ac_sub_evidence = len(civic_evidence_ids)
f"Total Number of accepted and submitted evidence items in CIViC: {total_ac_sub_evidence}"

'Total Number of accepted and submitted evidence items in CIViC: 9772'

## Total Molecular Profiles in CIViC

In [6]:
civic_molprof_ids = civicpy.get_all_molecular_profiles(include_status=["accepted", "submitted", "rejected"])

# Create analysis functions / global variables 

In [7]:
class VariantNormType(str, Enum):
  
    NORMALIZED = "Normalized"
    UNABLE_TO_NORMALIZE = "Unable to Normalize"
    NOT_SUPPORTED = "Not Supported"

VARIANT_NORM_TYPE_VALUES = [v.value for v in VariantNormType.__members__.values()]

In [8]:
class VariantCategory(str, Enum):
    """Create enum for the kind of variants that are in CIViC."""
    EXPRESSION = "Expression"
    EPIGENETIC_MODIFICATION = "Epigenetic Modification"
    FUSION = "Fusion"
    PROTEIN_CONS = "Protein Consequence"
    GENE_FUNC = "Gene Function"
    REARRANGEMENTS = "Rearrangements"
    COPY_NUMBER = "Copy Number"
    OTHER = "Other"
    GENOTYPES_EASY = "Genotypes Easy"
    GENOTYPES_COMPOUND = "Genotypes Compound"
    REGION_DEFINED_VAR = "Region Defined Variant"
    TRANSCRIPT_VAR = "Transcript Variant"  # no attempt to normalize these ones, since there is no query we could use

VARIANT_CATEGORY_VALUES = [v.value for v in VariantCategory.__members__.values()]

## Summary dicts 

These dictionaries will be mutated and used at the end of the analysis

In [9]:
VARIANT_ANALYSIS_SUMMARY = {
    "Variant Category": VARIANT_NORM_TYPE_VALUES,
    "Fraction of all CIViC Variant Items": [],
    "Percentage of all CIViC Variant Items": [],
    "Fraction of Accepted Variant Items": [],
    "Percentage of Accepted Variant Items":[],
    "Fraction of Not Accepted Variant Items": [],
    "Percentage of Not Accepted Variant Items": []
}
VARIANT_ANALYSIS_SUMMARY

{'Variant Category': ['Normalized', 'Unable to Normalize', 'Not Supported'],
 'Fraction of all CIViC Variant Items': [],
 'Percentage of all CIViC Variant Items': [],
 'Fraction of Accepted Variant Items': [],
 'Percentage of Accepted Variant Items': [],
 'Fraction of Not Accepted Variant Items': [],
 'Percentage of Not Accepted Variant Items': []}

In [10]:
EVIDENCE_ANALYSIS_SUMMARY = {
    "Variant Category": VARIANT_NORM_TYPE_VALUES,
    "Fraction of all CIViC Evidence Items": [],
    "Percentage of all CIViC Evidence Items": [],
    "Fraction of Accepted Evidence Items": [],
    "Percentage of Accepted Evidence Items":[],
    "Fraction of Submitted Evidence Items": [],
    "Percentage of Submitted Evidence Items": []
}
EVIDENCE_ANALYSIS_SUMMARY

{'Variant Category': ['Normalized', 'Unable to Normalize', 'Not Supported'],
 'Fraction of all CIViC Evidence Items': [],
 'Percentage of all CIViC Evidence Items': [],
 'Fraction of Accepted Evidence Items': [],
 'Percentage of Accepted Evidence Items': [],
 'Fraction of Submitted Evidence Items': [],
 'Percentage of Submitted Evidence Items': []}

In [11]:
def variant_analysis(df: pd.DataFrame, variant_norm_type: VariantNormType) -> pd.DataFrame:
    """Do variant analysis (counts, percentages)

    :param df: Dataframe of variants
    :param variant_norm_type: The kind of variants that are in `df`
    :return: Transformed dataframe with variant ID duplicates dropped
    """
    # Drop duplicate rows
    df = df.drop_duplicates(subset=["variant_id"])
    variant_ids = list(df["variant_id"])

    # Count
    num_variants = len(variant_ids)
    fraction_variants = f"{num_variants} / {total_number_variants}"
    print(f"\nNumber of {variant_norm_type.value} Variants in CIViC: {fraction_variants}")

    # Percentage
    percentage_variants = f"{num_variants / total_number_variants * 100:.2f}%"
    print(f"Percentage of {variant_norm_type.value} Variants in CIViC: {percentage_variants}")

    # Get accepted counts 
    num_accepted_variants = df.variant_accepted.sum()
    fraction_accepted_variants = f"{num_accepted_variants} / {num_variants}"
    print(f"\nNumber of accepted {variant_norm_type.value} Variants: {fraction_accepted_variants}")

    # Get accepted percentage
    percentage_accepted_variants = f"{num_accepted_variants / num_variants * 100:.2f}%"
    print(f"Percentage of accepted {variant_norm_type.value} Variants: {percentage_accepted_variants}")

    # Get not accepted counts 
    num_not_accepted_variants = len(df) - num_accepted_variants
    fraction_not_accepted_variants = f"{num_not_accepted_variants} / {num_variants}"
    print(f"\nNumber of not accepted {variant_norm_type.value} Variants: {fraction_not_accepted_variants}")

    # Get not accepted percentage
    percentage_not_accepted_variants = f"{num_not_accepted_variants / num_variants * 100:.2f}%"
    print(f"Percentage of not accepted {variant_norm_type.value} Variants: {percentage_not_accepted_variants}")
    print(variant_norm_type.name, variant_norm_type.value, variant_norm_type)

    VARIANT_ANALYSIS_SUMMARY["Fraction of all CIViC Variant Items"].append(fraction_variants)
    VARIANT_ANALYSIS_SUMMARY["Percentage of all CIViC Variant Items"].append(percentage_variants)
    VARIANT_ANALYSIS_SUMMARY["Fraction of Accepted Variant Items"].append(fraction_accepted_variants)
    VARIANT_ANALYSIS_SUMMARY["Percentage of Accepted Variant Items"].append(percentage_accepted_variants)
    VARIANT_ANALYSIS_SUMMARY["Fraction of Not Accepted Variant Items"].append(fraction_not_accepted_variants)
    VARIANT_ANALYSIS_SUMMARY["Percentage of Not Accepted Variant Items"].append(percentage_not_accepted_variants)

    return df

In [12]:
def transform_df_evidence_ids(df: pd.DataFrame) -> pd.DataFrame:
    """Transform dataframe to include evidence ID information

    :param df: Dataframe of variants
    :return: Transformed dataframe with evidence ID information
    """
    tmp_df = df.copy(deep=True)

    _variants_evidence_ids = []
    variant_ids = list(tmp_df["variant_id"])

    for v_id in variant_ids:
        _variant_evidence_ids = []
        
        for variant in civic_variant_ids:
            if int(v_id) == variant.id:
                for mp in variant.molecular_profiles:
                    for e in mp.evidence_items:
                        if e.id not in _variant_evidence_ids:
                            _variant_evidence_ids.append(e.id)

        _variants_evidence_ids.append(_variant_evidence_ids or "")
        
    tmp_df["evidence_ids"] = _variants_evidence_ids

    # Explode and rename evidence ids field 
    tmp_df = tmp_df.explode(column="evidence_ids")
    tmp_df = tmp_df.rename(columns={"evidence_ids": "evidence_id"})
    
    return tmp_df

In [13]:
def transform_df_evidence(df: pd.DataFrame) -> pd.DataFrame:
    """Transform dataframe to include evidence status, rating, and level

    :param df: Dataframe of variants
    :return: Transformed dataframe with evidence status, rating, and level information.
        Rejected evidence items will be dropped.
    """
    variants_evidence_ids = list(df["evidence_id"])

    # Add evidence status, rating, and level information
    _variants_evidence_statuses = []
    _variants_evidence_ratings = []
    _variants_evidence_levels = []

    for eid in variants_evidence_ids:
        _variant_evidence_statuses = []
        _variant_evidence_ratings = []
        _variant_evidence_levels = []

        for evidence in civic_evidence_ids: 
            if int(eid) == evidence.id:
                if evidence.status not in _variant_evidence_statuses:
                    _variant_evidence_statuses.append(evidence.status)

                if evidence.rating not in _variant_evidence_ratings:
                    _variant_evidence_ratings.append(evidence.rating)
                
                if evidence.evidence_level not in _variant_evidence_levels:
                    _variant_evidence_levels.append(evidence.evidence_level)
        
        _variants_evidence_statuses.append(_variant_evidence_statuses or "")
        _variants_evidence_ratings.append(_variant_evidence_ratings or "")
        _variants_evidence_levels.append(_variant_evidence_levels or "")

    df["evidence_status"] = _variants_evidence_statuses
    df["evidence_status"] = df["evidence_status"].str.join(", ")
    df["evidence_rating"] = _variants_evidence_ratings
    df["evidence_level"] = _variants_evidence_levels

    # Drop rejected evidence items
    df = df.drop(df[df.evidence_status == "rejected"].index)

    return df


In [14]:
def evidence_analysis(df: pd.DataFrame, variant_norm_type: VariantNormType) -> pd.DataFrame:
    """Do evidence analysis (counts, percentages)

    :param df: Dataframe of variants
    :param variant_norm_type: The kind of variants that are in `df`
    :return: Transformed dataframe with evidence ID duplicates dropped
    """
    # Count
    num_variant_unique_evidence_items = len(set(df.evidence_id))
    fraction_evidence_items = f"{num_variant_unique_evidence_items} / {total_ac_sub_evidence}"
    print(f"Number of {variant_norm_type.value} Variant Evidence items in CIViC: {fraction_evidence_items}")

    # Percentage
    percentage_evidence_items = f"{num_variant_unique_evidence_items / total_ac_sub_evidence * 100:.2f}%"
    print(f"Percentage of {variant_norm_type.value} Variant Evidence items in CIViC: {percentage_evidence_items}")

    # Add evidence accepted column
    df["evidence_accepted"] = df.evidence_status.map({"accepted": True , "submitted": False})

    # Drop evidence id duplicates
    df = df.drop_duplicates(subset=["evidence_id"])

    # Get accepted counts
    num_accepted_evidences_variants = df.evidence_accepted.sum()
    fraction_accepted_evidences_variants = f"{num_accepted_evidences_variants} / {num_variant_unique_evidence_items}"
    print(f"\nNumber of accepted {variant_norm_type.value} Variant Evidence items: {fraction_accepted_evidences_variants}")

    # Get accepted percentage
    percentage_accepted_evidences_variants = f"{num_accepted_evidences_variants / num_variant_unique_evidence_items * 100:.2f}%"
    print(f"Percentage of accepted {variant_norm_type.value} Variant Evidence items: {percentage_accepted_evidences_variants}")

    # Get submitted counts
    number_submitted_evidences_variants = len(df) - num_accepted_evidences_variants
    fraction_submitted_evidences_variants = f"{number_submitted_evidences_variants} / {num_variant_unique_evidence_items}"
    print(f"\nNumber of submitted {variant_norm_type.value} Variant Evidence items: {fraction_submitted_evidences_variants}")

    # Get submitted percentage
    percentage_submitted_evidences_variants = f"{number_submitted_evidences_variants / num_variant_unique_evidence_items * 100:.2f}%"
    print(f"Percentage of not submitted {variant_norm_type.value} Variant Evidence items: {percentage_submitted_evidences_variants}")

    EVIDENCE_ANALYSIS_SUMMARY["Fraction of all CIViC Evidence Items"].append(fraction_evidence_items)
    EVIDENCE_ANALYSIS_SUMMARY["Percentage of all CIViC Evidence Items"].append(percentage_evidence_items)
    EVIDENCE_ANALYSIS_SUMMARY["Fraction of Accepted Evidence Items"].append(fraction_accepted_evidences_variants)
    EVIDENCE_ANALYSIS_SUMMARY["Percentage of Accepted Evidence Items"].append(percentage_accepted_evidences_variants)
    EVIDENCE_ANALYSIS_SUMMARY["Fraction of Submitted Evidence Items"].append(fraction_submitted_evidences_variants)
    EVIDENCE_ANALYSIS_SUMMARY["Percentage of Submitted Evidence Items"].append(percentage_submitted_evidences_variants)

    return df

In [15]:
def transform_df_mp_id(df: pd.DataFrame) -> pd.DataFrame:
    """Transform dataframe to include molecular profile ID information

    :param df: Dataframe of variants
    :return: Transformed dataframe with molecular profile ID information
    """
    tmp_df = df.copy(deep=True)

    variants_molprof_ids = []
    variant_ids = list(tmp_df["variant_id"])

    for v_id in variant_ids:
        variant_molprof_ids = []

        for variant in civic_variant_ids:
            if int(v_id) == variant.id:
                for mp in variant.molecular_profiles:
                        if mp.id not in variant_molprof_ids:
                            variant_molprof_ids.append(mp.id)

        variants_molprof_ids.append(variant_molprof_ids or "")
        
    tmp_df["molecular_profile_id"] = variants_molprof_ids
    return tmp_df

In [16]:
def transform_df_mp_score(df: pd.DataFrame) -> pd.DataFrame:
    """Transform dataframe to include molecular profile score information

    :param df: Dataframe of variants
    :return: Transformed dataframe with molecular profile score information
    """
    variants_molprof_scores = []
    normalized_variant_molprof_ids = list(df["molecular_profile_id"])

    for mp_ids in normalized_variant_molprof_ids:
        variant_molprof_scores = []
        for mp_id in mp_ids:
            for molprof in civic_molprof_ids: 
                if int(mp_id) == molprof.id:
                    if molprof.molecular_profile_score not in variant_molprof_scores:
                        variant_molprof_scores.append(molprof.molecular_profile_score)
                    
        variants_molprof_scores.append(variant_molprof_scores or "")

    df["molecular_profile_score"] = variants_molprof_scores
    return df

In [17]:
def transform_df_mp_score_sum(df: pd.DataFrame) -> pd.DataFrame:
    """Transform dataframe to include molecular profile score sum information

    :param df: Dataframe of variants
    :return: Transformed dataframe with molecular profile score sum information
    """
    df["molecular_profile_score_sum"] = df["molecular_profile_score"].apply(lambda x: sum(x))
    return df

## List of Normalized Variants ID's

In [18]:
normalized_queries_df = pd.read_csv("able_to_normalize_queries.csv", sep="\t")
normalized_queries_df.head()

,variant_id,query,query_type,variant_accepted,vrs_id
0,2489,NC_000003.11:g.10191648_10191649insC,genomic,True,ga4gh:VA.A34ZoIhq4xBuQbcE3bkj29n6diS6RzLB
1,1988,NC_000003.11:g.10191649A>T,genomic,True,ga4gh:VA.JcEpDvhUtgDWU4A-bxqLUuczBNb8QqRf
2,2488,3-10191647-T-G,genomic,True,ga4gh:VA.7nGd8dgHbqtxMHk_rLxrB6_IMAzJ8XnH
3,1986,NC_000003.11:g.10191648G>T,genomic,True,ga4gh:VA.AmLtooLEvgdnEHD5YVWk6u1e2XBe7FiP
4,1987,NC_000003.11:g.10191649A>G,genomic,True,ga4gh:VA.KIz00usFWEmJHNyqmVL61obfgfRPgOIa


## Variant analysis

In [19]:
normalized_queries_df = variant_analysis(normalized_queries_df, VariantNormType.NORMALIZED)
normalized_queries_df.head()
    


Number of Normalized Variants in CIViC: 1869 / 3554
Percentage of Normalized Variants in CIViC: 52.59%

Number of accepted Normalized Variants: 866 / 1869
Percentage of accepted Normalized Variants: 46.33%

Number of not accepted Normalized Variants: 1003 / 1869
Percentage of not accepted Normalized Variants: 53.67%
NORMALIZED Normalized VariantNormType.NORMALIZED


,variant_id,query,query_type,variant_accepted,vrs_id
0,2489,NC_000003.11:g.10191648_10191649insC,genomic,True,ga4gh:VA.A34ZoIhq4xBuQbcE3bkj29n6diS6RzLB
1,1988,NC_000003.11:g.10191649A>T,genomic,True,ga4gh:VA.JcEpDvhUtgDWU4A-bxqLUuczBNb8QqRf
2,2488,3-10191647-T-G,genomic,True,ga4gh:VA.7nGd8dgHbqtxMHk_rLxrB6_IMAzJ8XnH
3,1986,NC_000003.11:g.10191648G>T,genomic,True,ga4gh:VA.AmLtooLEvgdnEHD5YVWk6u1e2XBe7FiP
4,1987,NC_000003.11:g.10191649A>G,genomic,True,ga4gh:VA.KIz00usFWEmJHNyqmVL61obfgfRPgOIa


In [20]:
VARIANT_ANALYSIS_SUMMARY

{'Variant Category': ['Normalized', 'Unable to Normalize', 'Not Supported'],
 'Fraction of all CIViC Variant Items': ['1869 / 3554'],
 'Percentage of all CIViC Variant Items': ['52.59%'],
 'Fraction of Accepted Variant Items': ['866 / 1869'],
 'Percentage of Accepted Variant Items': ['46.33%'],
 'Fraction of Not Accepted Variant Items': ['1003 / 1869'],
 'Percentage of Not Accepted Variant Items': ['53.67%']}

## Transform df for evidence analysis

In [21]:
normalized_queries_add_evidence_df = transform_df_evidence_ids(normalized_queries_df)
normalized_queries_add_evidence_df.head()

,variant_id,query,query_type,variant_accepted,vrs_id,evidence_id
0,2489,NC_000003.11:g.10191648_10191649insC,genomic,True,ga4gh:VA.A34ZoIhq4xBuQbcE3bkj29n6diS6RzLB,9347
0,2489,NC_000003.11:g.10191648_10191649insC,genomic,True,ga4gh:VA.A34ZoIhq4xBuQbcE3bkj29n6diS6RzLB,6724
1,1988,NC_000003.11:g.10191649A>T,genomic,True,ga4gh:VA.JcEpDvhUtgDWU4A-bxqLUuczBNb8QqRf,5336
2,2488,3-10191647-T-G,genomic,True,ga4gh:VA.7nGd8dgHbqtxMHk_rLxrB6_IMAzJ8XnH,10779
2,2488,3-10191647-T-G,genomic,True,ga4gh:VA.7nGd8dgHbqtxMHk_rLxrB6_IMAzJ8XnH,6723


In [22]:
normalized_queries_add_evidence_df = transform_df_evidence(normalized_queries_add_evidence_df)
normalized_queries_add_evidence_df.head()

,variant_id,query,query_type,variant_accepted,vrs_id,evidence_id,evidence_status,evidence_rating,evidence_level
0,2489,NC_000003.11:g.10191648_10191649insC,genomic,True,ga4gh:VA.A34ZoIhq4xBuQbcE3bkj29n6diS6RzLB,9347,submitted,[3],[C]
0,2489,NC_000003.11:g.10191648_10191649insC,genomic,True,ga4gh:VA.A34ZoIhq4xBuQbcE3bkj29n6diS6RzLB,6724,accepted,[2],[C]
1,1988,NC_000003.11:g.10191649A>T,genomic,True,ga4gh:VA.JcEpDvhUtgDWU4A-bxqLUuczBNb8QqRf,5336,accepted,[2],[C]
2,2488,3-10191647-T-G,genomic,True,ga4gh:VA.7nGd8dgHbqtxMHk_rLxrB6_IMAzJ8XnH,10779,submitted,[3],[C]
2,2488,3-10191647-T-G,genomic,True,ga4gh:VA.7nGd8dgHbqtxMHk_rLxrB6_IMAzJ8XnH,6723,accepted,[2],[C]


## Evidence analysis

In [23]:
normalized_queries_add_evidence_df = evidence_analysis(normalized_queries_add_evidence_df, VariantNormType.NORMALIZED)
normalized_queries_add_evidence_df.head()

Number of Normalized Variant Evidence items in CIViC: 5917 / 9772
Percentage of Normalized Variant Evidence items in CIViC: 60.55%

Number of accepted Normalized Variant Evidence items: 2033 / 5917
Percentage of accepted Normalized Variant Evidence items: 34.36%

Number of submitted Normalized Variant Evidence items: 3884 / 5917
Percentage of not submitted Normalized Variant Evidence items: 65.64%


,variant_id,query,query_type,variant_accepted,vrs_id,evidence_id,evidence_status,evidence_rating,evidence_level,evidence_accepted
0,2489,NC_000003.11:g.10191648_10191649insC,genomic,True,ga4gh:VA.A34ZoIhq4xBuQbcE3bkj29n6diS6RzLB,9347,submitted,[3],[C],False
0,2489,NC_000003.11:g.10191648_10191649insC,genomic,True,ga4gh:VA.A34ZoIhq4xBuQbcE3bkj29n6diS6RzLB,6724,accepted,[2],[C],True
1,1988,NC_000003.11:g.10191649A>T,genomic,True,ga4gh:VA.JcEpDvhUtgDWU4A-bxqLUuczBNb8QqRf,5336,accepted,[2],[C],True
2,2488,3-10191647-T-G,genomic,True,ga4gh:VA.7nGd8dgHbqtxMHk_rLxrB6_IMAzJ8XnH,10779,submitted,[3],[C],False
2,2488,3-10191647-T-G,genomic,True,ga4gh:VA.7nGd8dgHbqtxMHk_rLxrB6_IMAzJ8XnH,6723,accepted,[2],[C],True


## Impact
molecular profile score

### Import molecular profile id

In [24]:
normalized_queries_add_molprof_df = transform_df_mp_id(normalized_queries_df)
normalized_queries_add_molprof_df.head()

,variant_id,query,query_type,variant_accepted,vrs_id,molecular_profile_id
0,2489,NC_000003.11:g.10191648_10191649insC,genomic,True,ga4gh:VA.A34ZoIhq4xBuQbcE3bkj29n6diS6RzLB,[2362]
1,1988,NC_000003.11:g.10191649A>T,genomic,True,ga4gh:VA.JcEpDvhUtgDWU4A-bxqLUuczBNb8QqRf,[1864]
2,2488,3-10191647-T-G,genomic,True,ga4gh:VA.7nGd8dgHbqtxMHk_rLxrB6_IMAzJ8XnH,[2361]
3,1986,NC_000003.11:g.10191648G>T,genomic,True,ga4gh:VA.AmLtooLEvgdnEHD5YVWk6u1e2XBe7FiP,[1862]
4,1987,NC_000003.11:g.10191649A>G,genomic,True,ga4gh:VA.KIz00usFWEmJHNyqmVL61obfgfRPgOIa,[1863]


In [25]:
normalized_queries_add_molprof_df.loc[normalized_queries_add_molprof_df["variant_id"] == 190 ]

,variant_id,query,query_type,variant_accepted,vrs_id,molecular_profile_id
82,190,EGFR Amplification,protein,True,ga4gh:CX.sEHT64Lm86QaTXzw39uKLkBUbEkp4h_X,"[190, 4175, 4346]"


### Import molecular profile scores

In [26]:
normalized_queries_add_molprof_df = transform_df_mp_score(normalized_queries_add_molprof_df)
normalized_queries_add_molprof_df.head()

,variant_id,query,query_type,variant_accepted,vrs_id,molecular_profile_id,molecular_profile_score
0,2489,NC_000003.11:g.10191648_10191649insC,genomic,True,ga4gh:VA.A34ZoIhq4xBuQbcE3bkj29n6diS6RzLB,[2362],[5.0]
1,1988,NC_000003.11:g.10191649A>T,genomic,True,ga4gh:VA.JcEpDvhUtgDWU4A-bxqLUuczBNb8QqRf,[1864],[5.0]
2,2488,3-10191647-T-G,genomic,True,ga4gh:VA.7nGd8dgHbqtxMHk_rLxrB6_IMAzJ8XnH,[2361],[5.0]
3,1986,NC_000003.11:g.10191648G>T,genomic,True,ga4gh:VA.AmLtooLEvgdnEHD5YVWk6u1e2XBe7FiP,[1862],[10.0]
4,1987,NC_000003.11:g.10191649A>G,genomic,True,ga4gh:VA.KIz00usFWEmJHNyqmVL61obfgfRPgOIa,[1863],[5.0]


In [27]:
normalized_queries_add_molprof_df.loc[normalized_queries_add_molprof_df["variant_id"] == 190 ]

,variant_id,query,query_type,variant_accepted,vrs_id,molecular_profile_id,molecular_profile_score
82,190,EGFR Amplification,protein,True,ga4gh:CX.sEHT64Lm86QaTXzw39uKLkBUbEkp4h_X,"[190, 4175, 4346]","[173.0, 5.0, 0.0]"


In [28]:
normalized_queries_add_molprof_df = transform_df_mp_score_sum(normalized_queries_add_molprof_df)
normalized_queries_add_molprof_df.head()

,variant_id,query,query_type,variant_accepted,vrs_id,molecular_profile_id,molecular_profile_score,molecular_profile_score_sum
0,2489,NC_000003.11:g.10191648_10191649insC,genomic,True,ga4gh:VA.A34ZoIhq4xBuQbcE3bkj29n6diS6RzLB,[2362],[5.0],5.0
1,1988,NC_000003.11:g.10191649A>T,genomic,True,ga4gh:VA.JcEpDvhUtgDWU4A-bxqLUuczBNb8QqRf,[1864],[5.0],5.0
2,2488,3-10191647-T-G,genomic,True,ga4gh:VA.7nGd8dgHbqtxMHk_rLxrB6_IMAzJ8XnH,[2361],[5.0],5.0
3,1986,NC_000003.11:g.10191648G>T,genomic,True,ga4gh:VA.AmLtooLEvgdnEHD5YVWk6u1e2XBe7FiP,[1862],[10.0],10.0
4,1987,NC_000003.11:g.10191649A>G,genomic,True,ga4gh:VA.KIz00usFWEmJHNyqmVL61obfgfRPgOIa,[1863],[5.0],5.0


In [29]:
normalized_queries_add_molprof_df.loc[normalized_queries_add_molprof_df["variant_id"] == 190 ]

,variant_id,query,query_type,variant_accepted,vrs_id,molecular_profile_id,molecular_profile_score,molecular_profile_score_sum
82,190,EGFR Amplification,protein,True,ga4gh:CX.sEHT64Lm86QaTXzw39uKLkBUbEkp4h_X,"[190, 4175, 4346]","[173.0, 5.0, 0.0]",178.0


# Analysis of Unable to Normalize Queries

## List of Unable to Normalize Variant ID's

In [30]:
not_normalized_queries_df = pd.read_csv("unable_to_normalize_queries.csv", sep= "\t")
not_normalized_queries_df.head()

,variant_id,query,query_type,variant_accepted,exception_raised,message,warnings
0,748,MLH1 *757L,protein,True,False,unable to normalize,"[""Unable to find valid result for classificati..."
1,3718,AR A748V,protein,False,False,unable to normalize,"[""Unable to find valid result for classificati..."
2,3725,AR A765T,protein,False,False,unable to normalize,"[""Unable to find valid result for classificati..."
3,248,TERT C228T,protein,True,False,unable to normalize,"[""Unable to find valid result for classificati..."
4,4004,TERT C250T,protein,False,False,unable to normalize,"[""Unable to find valid result for classificati..."


## Variant analysis

In [31]:
not_normalized_queries_df = variant_analysis(not_normalized_queries_df, VariantNormType.UNABLE_TO_NORMALIZE)
not_normalized_queries_df.head()


Number of Unable to Normalize Variants in CIViC: 63 / 3554
Percentage of Unable to Normalize Variants in CIViC: 1.77%

Number of accepted Unable to Normalize Variants: 9 / 63
Percentage of accepted Unable to Normalize Variants: 14.29%

Number of not accepted Unable to Normalize Variants: 54 / 63
Percentage of not accepted Unable to Normalize Variants: 85.71%
UNABLE_TO_NORMALIZE Unable to Normalize VariantNormType.UNABLE_TO_NORMALIZE


,variant_id,query,query_type,variant_accepted,exception_raised,message,warnings
0,748,MLH1 *757L,protein,True,False,unable to normalize,"[""Unable to find valid result for classificati..."
1,3718,AR A748V,protein,False,False,unable to normalize,"[""Unable to find valid result for classificati..."
2,3725,AR A765T,protein,False,False,unable to normalize,"[""Unable to find valid result for classificati..."
3,248,TERT C228T,protein,True,False,unable to normalize,"[""Unable to find valid result for classificati..."
4,4004,TERT C250T,protein,False,False,unable to normalize,"[""Unable to find valid result for classificati..."


## Transform df for evidence analysis

In [32]:
not_normalized_quer_add_evidence_df = transform_df_evidence_ids(not_normalized_queries_df)
not_normalized_quer_add_evidence_df.head()

,variant_id,query,query_type,variant_accepted,exception_raised,message,warnings,evidence_id
0,748,MLH1 *757L,protein,True,False,unable to normalize,"[""Unable to find valid result for classificati...",1812
1,3718,AR A748V,protein,False,False,unable to normalize,"[""Unable to find valid result for classificati...",10128
2,3725,AR A765T,protein,False,False,unable to normalize,"[""Unable to find valid result for classificati...",10135
3,248,TERT C228T,protein,True,False,unable to normalize,"[""Unable to find valid result for classificati...",655
3,248,TERT C228T,protein,True,False,unable to normalize,"[""Unable to find valid result for classificati...",1646


In [33]:
not_normalized_quer_add_evidence_df = transform_df_evidence(not_normalized_quer_add_evidence_df)
not_normalized_quer_add_evidence_df.head()

,variant_id,query,query_type,variant_accepted,exception_raised,message,warnings,evidence_id,evidence_status,evidence_rating,evidence_level
0,748,MLH1 *757L,protein,True,False,unable to normalize,"[""Unable to find valid result for classificati...",1812,accepted,[1],[C]
1,3718,AR A748V,protein,False,False,unable to normalize,"[""Unable to find valid result for classificati...",10128,submitted,[3],[D]
2,3725,AR A765T,protein,False,False,unable to normalize,"[""Unable to find valid result for classificati...",10135,submitted,[3],[D]
3,248,TERT C228T,protein,True,False,unable to normalize,"[""Unable to find valid result for classificati...",655,accepted,[5],[B]
3,248,TERT C228T,protein,True,False,unable to normalize,"[""Unable to find valid result for classificati...",1646,accepted,[3],[B]


## Evidence analysis

In [34]:
not_normalized_quer_add_evidence_df = evidence_analysis(not_normalized_quer_add_evidence_df, VariantNormType.UNABLE_TO_NORMALIZE)
not_normalized_quer_add_evidence_df.head()

Number of Unable to Normalize Variant Evidence items in CIViC: 80 / 9772
Percentage of Unable to Normalize Variant Evidence items in CIViC: 0.82%

Number of accepted Unable to Normalize Variant Evidence items: 14 / 80
Percentage of accepted Unable to Normalize Variant Evidence items: 17.50%

Number of submitted Unable to Normalize Variant Evidence items: 66 / 80
Percentage of not submitted Unable to Normalize Variant Evidence items: 82.50%


,variant_id,query,query_type,variant_accepted,exception_raised,message,warnings,evidence_id,evidence_status,evidence_rating,evidence_level,evidence_accepted
0,748,MLH1 *757L,protein,True,False,unable to normalize,"[""Unable to find valid result for classificati...",1812,accepted,[1],[C],True
1,3718,AR A748V,protein,False,False,unable to normalize,"[""Unable to find valid result for classificati...",10128,submitted,[3],[D],False
2,3725,AR A765T,protein,False,False,unable to normalize,"[""Unable to find valid result for classificati...",10135,submitted,[3],[D],False
3,248,TERT C228T,protein,True,False,unable to normalize,"[""Unable to find valid result for classificati...",655,accepted,[5],[B],True
3,248,TERT C228T,protein,True,False,unable to normalize,"[""Unable to find valid result for classificati...",1646,accepted,[3],[B],True


## Impact
molecular profile score

### Import molecular profile id

In [35]:
not_normalized_queries_add_molprof_df = transform_df_mp_id(not_normalized_queries_df)
not_normalized_queries_add_molprof_df.head()

,variant_id,query,query_type,variant_accepted,exception_raised,message,warnings,molecular_profile_id
0,748,MLH1 *757L,protein,True,False,unable to normalize,"[""Unable to find valid result for classificati...",[729]
1,3718,AR A748V,protein,False,False,unable to normalize,"[""Unable to find valid result for classificati...",[3586]
2,3725,AR A765T,protein,False,False,unable to normalize,"[""Unable to find valid result for classificati...",[3593]
3,248,TERT C228T,protein,True,False,unable to normalize,"[""Unable to find valid result for classificati...",[244]
4,4004,TERT C250T,protein,False,False,unable to normalize,"[""Unable to find valid result for classificati...",[3872]


### Import molecular profile scores

In [36]:
not_normalized_queries_add_molprof_df = transform_df_mp_score(not_normalized_queries_add_molprof_df)
not_normalized_queries_add_molprof_df.head()

,variant_id,query,query_type,variant_accepted,exception_raised,message,warnings,molecular_profile_id,molecular_profile_score
0,748,MLH1 *757L,protein,True,False,unable to normalize,"[""Unable to find valid result for classificati...",[729],[2.5]
1,3718,AR A748V,protein,False,False,unable to normalize,"[""Unable to find valid result for classificati...",[3586],[0.0]
2,3725,AR A765T,protein,False,False,unable to normalize,"[""Unable to find valid result for classificati...",[3593],[0.0]
3,248,TERT C228T,protein,True,False,unable to normalize,"[""Unable to find valid result for classificati...",[244],[40.0]
4,4004,TERT C250T,protein,False,False,unable to normalize,"[""Unable to find valid result for classificati...",[3872],[0.0]


In [37]:
not_normalized_queries_add_molprof_df = transform_df_mp_score_sum(not_normalized_queries_add_molprof_df)
not_normalized_queries_add_molprof_df.head()

,variant_id,query,query_type,variant_accepted,exception_raised,message,warnings,molecular_profile_id,molecular_profile_score,molecular_profile_score_sum
0,748,MLH1 *757L,protein,True,False,unable to normalize,"[""Unable to find valid result for classificati...",[729],[2.5],2.5
1,3718,AR A748V,protein,False,False,unable to normalize,"[""Unable to find valid result for classificati...",[3586],[0.0],0.0
2,3725,AR A765T,protein,False,False,unable to normalize,"[""Unable to find valid result for classificati...",[3593],[0.0],0.0
3,248,TERT C228T,protein,True,False,unable to normalize,"[""Unable to find valid result for classificati...",[244],[40.0],40.0
4,4004,TERT C250T,protein,False,False,unable to normalize,"[""Unable to find valid result for classificati...",[3872],[0.0],0.0


# Analysis of Not Supported Variants

### List of Not Supported Variant ID's

In [38]:
not_supported_queries_df = pd.read_csv("not_supported_variants.csv", sep= "\t")
not_supported_queries_df.head()

,variant_id,gene_name,variant_name,category,variant_accepted
0,4170,VHL,NaN,Transcript Variant,False
1,4417,ALK,FBXO11::ALK,Fusion,False
2,4214,VHL,NaN,Transcript Variant,False
3,4216,VHL,NaN,Transcript Variant,False
4,4278,VHL,NaN,Transcript Variant,False


## Variant Analysis

In [39]:
not_supported_queries_df = variant_analysis(not_supported_queries_df, VariantNormType.NOT_SUPPORTED)
not_supported_queries_df.head()
    


Number of Not Supported Variants in CIViC: 1622 / 3554
Percentage of Not Supported Variants in CIViC: 45.64%

Number of accepted Not Supported Variants: 788 / 1622
Percentage of accepted Not Supported Variants: 48.58%

Number of not accepted Not Supported Variants: 834 / 1622
Percentage of not accepted Not Supported Variants: 51.42%
NOT_SUPPORTED Not Supported VariantNormType.NOT_SUPPORTED


,variant_id,gene_name,variant_name,category,variant_accepted
0,4170,VHL,NaN,Transcript Variant,False
1,4417,ALK,FBXO11::ALK,Fusion,False
2,4214,VHL,NaN,Transcript Variant,False
3,4216,VHL,NaN,Transcript Variant,False
4,4278,VHL,NaN,Transcript Variant,False


### Not Supported Variant Analysis by Subcategory 

In [40]:
not_supported_variant_categories_summary_data = dict()
total_number_unique_not_supported_variants = len(set(not_supported_queries_df.variant_id))
for category in VARIANT_CATEGORY_VALUES:
    print(category)
    not_supported_variant_categories_summary_data[category] = {}
    category_df = not_supported_queries_df[not_supported_queries_df.category == category]
    
    number_unique_not_supported_category_variants = len(set(category_df.variant_id))
    not_supported_variant_categories_summary_data[category]["number_unique_not_supported_category_variants"] = number_unique_not_supported_category_variants

    fraction_not_supported_category_variant_of_civic = f" {number_unique_not_supported_category_variants} / {total_number_variants}"
    not_supported_variant_categories_summary_data[category]["fraction_not_supported_category_variant_of_civic"] = fraction_not_supported_category_variant_of_civic
    print(f"Number of {category} Variants in CIViC: {fraction_not_supported_category_variant_of_civic}")
    
    percent_not_supported_category_variant_of_civic = "{:.2f}".format(number_unique_not_supported_category_variants/total_number_variants*100) + "%"
    not_supported_variant_categories_summary_data[category]["percent_not_supported_category_variant_of_civic"] = percent_not_supported_category_variant_of_civic
    print(f"Percent of {category} Variants in CIViC: {percent_not_supported_category_variant_of_civic}")

    fraction_not_supported_category_variant_of_total_not_supported = f" {number_unique_not_supported_category_variants} / {total_number_unique_not_supported_variants}"
    not_supported_variant_categories_summary_data[category]["fraction_not_supported_category_variant_of_total_not_supported"] = fraction_not_supported_category_variant_of_total_not_supported
    print(f"Number of {category} Variants in Not Supported Variants: {fraction_not_supported_category_variant_of_total_not_supported}")
    
    percent_not_supported_category_variant_of_total_not_supported = "{:.2f}".format(number_unique_not_supported_category_variants/total_number_unique_not_supported_variants*100) + "%"
    not_supported_variant_categories_summary_data[category]["percent_not_supported_category_variant_of_total_not_supported"] = percent_not_supported_category_variant_of_total_not_supported
    print(f"Percent of {category} Variants in Not Supported Variants: {percent_not_supported_category_variant_of_total_not_supported}")

    number_accepted_not_supported_category_variants = category_df.variant_accepted.sum()
    fraction_accepted_not_supported_category_variants = f" {number_accepted_not_supported_category_variants} / {number_unique_not_supported_category_variants}"
    not_supported_variant_categories_summary_data[category]["fraction_accepted_not_supported_category_variants"] = fraction_accepted_not_supported_category_variants
    print(f"Number of Accepted {category} Variants: {fraction_accepted_not_supported_category_variants}")
   
    percentage_accepted_not_supported_category_variants =  "{:.2f}".format(number_accepted_not_supported_category_variants/number_unique_not_supported_category_variants*100) + "%"
    not_supported_variant_categories_summary_data[category]["percentage_accepted_not_supported_category_variants"] = percentage_accepted_not_supported_category_variants
    print(f"Percent of Accepted {category} Variants: {percentage_accepted_not_supported_category_variants}")
  
    number_not_accepted_not_supported_category_variants =  len(category_df) - category_df.variant_accepted.sum()
    fraction_not_accepted_not_supported_category_variants = f" {number_not_accepted_not_supported_category_variants} / {number_unique_not_supported_category_variants}"
    not_supported_variant_categories_summary_data[category]["fraction_not_accepted_not_supported_category_variants"] = fraction_not_accepted_not_supported_category_variants
    print(f"Number of Not Accepted {category} Variants: {fraction_not_accepted_not_supported_category_variants}")

    percentage_not_accepted_not_supported_category_variants =  "{:.2f}".format(number_not_accepted_not_supported_category_variants/number_unique_not_supported_category_variants*100) + "%"
    not_supported_variant_categories_summary_data[category]["percentage_not_accepted_not_supported_category_variants"] = percentage_not_accepted_not_supported_category_variants
    print(f"Percent of Not Accepted {category} Variants: {percentage_not_accepted_not_supported_category_variants}")
    print("--------------------")

Expression
Number of Expression Variants in CIViC:  291 / 3554
Percent of Expression Variants in CIViC: 8.19%
Number of Expression Variants in Not Supported Variants:  291 / 1622
Percent of Expression Variants in Not Supported Variants: 17.94%
Number of Accepted Expression Variants:  180 / 291
Percent of Accepted Expression Variants: 61.86%
Number of Not Accepted Expression Variants:  111 / 291
Percent of Not Accepted Expression Variants: 38.14%
--------------------
Epigenetic Modification
Number of Epigenetic Modification Variants in CIViC:  15 / 3554
Percent of Epigenetic Modification Variants in CIViC: 0.42%
Number of Epigenetic Modification Variants in Not Supported Variants:  15 / 1622
Percent of Epigenetic Modification Variants in Not Supported Variants: 0.92%
Number of Accepted Epigenetic Modification Variants:  14 / 15
Percent of Accepted Epigenetic Modification Variants: 93.33%
Number of Not Accepted Epigenetic Modification Variants:  1 / 15
Percent of Not Accepted Epigenetic 

## Transform df for evidence analysis

In [41]:
not_supported_variants_add_evidence_df = transform_df_evidence_ids(not_supported_queries_df)
not_supported_variants_add_evidence_df.head()

,variant_id,gene_name,variant_name,category,variant_accepted,evidence_id
0,4170,VHL,NaN,Transcript Variant,False,10647
1,4417,ALK,FBXO11::ALK,Fusion,False,7428
2,4214,VHL,NaN,Transcript Variant,False,10752
3,4216,VHL,NaN,Transcript Variant,False,10754
4,4278,VHL,NaN,Transcript Variant,False,10958


In [42]:
not_supported_variants_add_evidence_df = transform_df_evidence(not_supported_variants_add_evidence_df)
not_supported_variants_add_evidence_df.head()

,variant_id,gene_name,variant_name,category,variant_accepted,evidence_id,evidence_status,evidence_rating,evidence_level
0,4170,VHL,NaN,Transcript Variant,False,10647,submitted,[2],[C]
1,4417,ALK,FBXO11::ALK,Fusion,False,7428,submitted,[3],[C]
2,4214,VHL,NaN,Transcript Variant,False,10752,submitted,[3],[C]
3,4216,VHL,NaN,Transcript Variant,False,10754,submitted,[3],[C]
4,4278,VHL,NaN,Transcript Variant,False,10958,submitted,[3],[C]


## Evidence analysis

In [43]:
not_supported_variants_add_evidence_df = evidence_analysis(not_supported_variants_add_evidence_df, VariantNormType.NOT_SUPPORTED)
not_supported_variants_add_evidence_df.head()

Number of Not Supported Variant Evidence items in CIViC: 4293 / 9772
Percentage of Not Supported Variant Evidence items in CIViC: 43.93%

Number of accepted Not Supported Variant Evidence items: 2201 / 4293
Percentage of accepted Not Supported Variant Evidence items: 51.27%

Number of submitted Not Supported Variant Evidence items: 2092 / 4293
Percentage of not submitted Not Supported Variant Evidence items: 48.73%


,variant_id,gene_name,variant_name,category,variant_accepted,evidence_id,evidence_status,evidence_rating,evidence_level,evidence_accepted
0,4170,VHL,NaN,Transcript Variant,False,10647,submitted,[2],[C],False
1,4417,ALK,FBXO11::ALK,Fusion,False,7428,submitted,[3],[C],False
2,4214,VHL,NaN,Transcript Variant,False,10752,submitted,[3],[C],False
3,4216,VHL,NaN,Transcript Variant,False,10754,submitted,[3],[C],False
4,4278,VHL,NaN,Transcript Variant,False,10958,submitted,[3],[C],False


### Not Supported Variant Evidence Analysis by Subcategory 

 List all the possible variant categories, have to use non unique file since evidence items are used more than once across groups


In [44]:
not_supported_variant_categories = not_supported_variants_add_evidence_df.category.unique()
not_supported_variant_categories 

array(['Transcript Variant', 'Fusion', 'Region Defined Variant',
       'Protein Consequence', 'Other', 'Rearrangements', 'Copy Number',
       'Expression', 'Gene Function', 'Genotypes Compound',
       'Epigenetic Modification', 'Genotypes Easy'], dtype=object)

Evidence ID"s can qualify for more than one kind of variant category

In [45]:
duplicate = not_supported_variants_add_evidence_df[not_supported_variants_add_evidence_df.duplicated('evidence_id', keep=False)]
duplicate

,variant_id,gene_name,variant_name,category,variant_accepted,evidence_id,evidence_status,evidence_rating,evidence_level,evidence_accepted


In [46]:
not_supported_variant_categories_evidence_summary_data = dict()
total_number_not_supported_variant_unique_evidence_items = len(set(not_supported_variants_add_evidence_df.evidence_id))
for category in VARIANT_CATEGORY_VALUES:
    print(category)
    not_supported_variant_categories_evidence_summary_data[category] = {}
    evidence_category_df = not_supported_variants_add_evidence_df[not_supported_variants_add_evidence_df.category == category]
    evidence_category_df = evidence_category_df.drop_duplicates(subset=['evidence_id'])

    number_unique_not_supported_category_evidence = len(set(evidence_category_df.evidence_id))
    not_supported_variant_categories_evidence_summary_data[category]["number_unique_not_supported_category_evidence"]=  number_unique_not_supported_category_evidence
    
    fraction_not_supported_category_variant_evidence_of_civic = f"{number_unique_not_supported_category_evidence} / {total_ac_sub_evidence}"
    not_supported_variant_categories_evidence_summary_data[category]["fraction_not_supported_category_variant_evidence_of_civic"]=  fraction_not_supported_category_variant_evidence_of_civic
    print(f"Number of {category} Variant Evidence items in CIViC: {fraction_not_supported_category_variant_evidence_of_civic}")

    percent_not_supported_category_variant_evidence_of_civic = "{:.2f}".format(number_unique_not_supported_category_evidence/total_ac_sub_evidence*100)  + '%'
    not_supported_variant_categories_evidence_summary_data[category]["percent_not_supported_category_variant_evidence_of_civic"] = percent_not_supported_category_variant_evidence_of_civic
    print(f"Percent of {category} Variant Evidence items in CIViC: {percent_not_supported_category_variant_evidence_of_civic}")
    
    fraction_not_supported_category_variant_evidence_of_total_not_supported = f"{number_unique_not_supported_category_evidence} / {total_number_not_supported_variant_unique_evidence_items}"
    not_supported_variant_categories_evidence_summary_data[category]["fraction_not_supported_category_variant_evidence_of_total_not_supported"]= fraction_not_supported_category_variant_evidence_of_total_not_supported
    print(f"Number of {category} Variant Evidence items in total Not Supported Variant Evidence items: {fraction_not_supported_category_variant_evidence_of_total_not_supported}")

    percent_not_supported_category_variant_evidence_of_total_not_supported = "{:.2f}".format(number_unique_not_supported_category_evidence/total_number_not_supported_variant_unique_evidence_items*100) + '%'
    not_supported_variant_categories_evidence_summary_data[category]["percent_not_supported_category_variant_evidence_of_total_not_supported"] = percent_not_supported_category_variant_evidence_of_total_not_supported
    print(f"Percent of {category} Variant Evidence items in total Not Supported Variant Evidence items: {percent_not_supported_category_variant_evidence_of_total_not_supported}")

    number_accepted_not_supported_category_variant_evidence = evidence_category_df.evidence_accepted.sum()
    fraction_accepted_evidence_not_supported_category_variants = f"{number_accepted_not_supported_category_variant_evidence} / {number_unique_not_supported_category_evidence}"
    not_supported_variant_categories_evidence_summary_data[category]["fraction_accepted_evidence_not_supported_category_variants"] = fraction_accepted_evidence_not_supported_category_variants
    print(f"Number of Accepted {category} Variant Evidence items: {fraction_accepted_evidence_not_supported_category_variants}")

    percentage_accepted_evidence_not_supported_category_variants =  "{:.2f}".format(number_accepted_not_supported_category_variant_evidence/number_unique_not_supported_category_evidence*100) + '%'
    not_supported_variant_categories_evidence_summary_data[category]["percentage_accepted_evidence_not_supported_category_variants"] = percentage_accepted_evidence_not_supported_category_variants
    print(f"Percent of Accepted {category} Variant Evidence items: {percentage_accepted_evidence_not_supported_category_variants}")
  
    number_submitted_not_supported_category_variant_evidence = number_unique_not_supported_category_evidence - evidence_category_df.evidence_accepted.sum()
    fraction_submitted_evidence_not_supported_category_variants = f"{number_submitted_not_supported_category_variant_evidence} / {number_unique_not_supported_category_evidence}"
    not_supported_variant_categories_evidence_summary_data[category]["fraction_submitted_evidence_not_supported_category_variants"] = fraction_submitted_evidence_not_supported_category_variants
    print(f"Number of submitted {category} Variant Evidence items: {fraction_submitted_evidence_not_supported_category_variants}")

    percentage_submitted_evidence_not_supported_category_variants =  "{:.2f}".format(number_submitted_not_supported_category_variant_evidence/number_unique_not_supported_category_evidence*100) + '%'
    not_supported_variant_categories_evidence_summary_data[category]["percentage_submitted_evidence_not_supported_category_variants"] = percentage_submitted_evidence_not_supported_category_variants
    print(f"Percent of submitted {category} Variant Evidence items: {percentage_submitted_evidence_not_supported_category_variants}")
    
    print("--------------------")


Expression
Number of Expression Variant Evidence items in CIViC: 614 / 9772
Percent of Expression Variant Evidence items in CIViC: 6.28%
Number of Expression Variant Evidence items in total Not Supported Variant Evidence items: 614 / 4293
Percent of Expression Variant Evidence items in total Not Supported Variant Evidence items: 14.30%
Number of Accepted Expression Variant Evidence items: 342 / 614
Percent of Accepted Expression Variant Evidence items: 55.70%
Number of submitted Expression Variant Evidence items: 272 / 614
Percent of submitted Expression Variant Evidence items: 44.30%
--------------------
Epigenetic Modification
Number of Epigenetic Modification Variant Evidence items in CIViC: 24 / 9772
Percent of Epigenetic Modification Variant Evidence items in CIViC: 0.25%
Number of Epigenetic Modification Variant Evidence items in total Not Supported Variant Evidence items: 24 / 4293
Percent of Epigenetic Modification Variant Evidence items in total Not Supported Variant Evidence 

## Impact
molecular profile score

### Import molecular profile id

In [47]:
not_supported_variants_add_molprof_df = transform_df_mp_id(not_supported_queries_df)
not_supported_variants_add_molprof_df.head()

,variant_id,gene_name,variant_name,category,variant_accepted,molecular_profile_id
0,4170,VHL,NaN,Transcript Variant,False,[4038]
1,4417,ALK,FBXO11::ALK,Fusion,False,[4350]
2,4214,VHL,NaN,Transcript Variant,False,[4082]
3,4216,VHL,NaN,Transcript Variant,False,[4084]
4,4278,VHL,NaN,Transcript Variant,False,[4146]


### Import molecular profile scores

In [48]:
not_supported_variants_add_molprof_df = transform_df_mp_score(not_supported_variants_add_molprof_df)
not_supported_variants_add_molprof_df.head()

,variant_id,gene_name,variant_name,category,variant_accepted,molecular_profile_id,molecular_profile_score
0,4170,VHL,NaN,Transcript Variant,False,[4038],[0.0]
1,4417,ALK,FBXO11::ALK,Fusion,False,[4350],[0.0]
2,4214,VHL,NaN,Transcript Variant,False,[4082],[0.0]
3,4216,VHL,NaN,Transcript Variant,False,[4084],[0.0]
4,4278,VHL,NaN,Transcript Variant,False,[4146],[0.0]


In [49]:
not_supported_variants_add_molprof_df = transform_df_mp_score_sum(not_supported_variants_add_molprof_df)
not_supported_variants_add_molprof_df.head()

,variant_id,gene_name,variant_name,category,variant_accepted,molecular_profile_id,molecular_profile_score,molecular_profile_score_sum
0,4170,VHL,NaN,Transcript Variant,False,[4038],[0.0],0.0
1,4417,ALK,FBXO11::ALK,Fusion,False,[4350],[0.0],0.0
2,4214,VHL,NaN,Transcript Variant,False,[4082],[0.0],0.0
3,4216,VHL,NaN,Transcript Variant,False,[4084],[0.0],0.0
4,4278,VHL,NaN,Transcript Variant,False,[4146],[0.0],0.0


### Impact by Subcategory

In [50]:
not_supported_variant_categories_impact_data = dict()
for category in VARIANT_CATEGORY_VALUES:
    not_supported_variant_categories_impact_data[category] = {}
    impact_category_df = not_supported_variants_add_molprof_df[not_supported_variants_add_molprof_df.category == category]

    total_sum_not_supported_category_impact = impact_category_df["molecular_profile_score_sum"].sum()
    not_supported_variant_categories_impact_data[category]["total_sum_not_supported_category_impact"]=  total_sum_not_supported_category_impact
    print(f"{category}: {total_sum_not_supported_category_impact}")

Expression: 3618.0
Epigenetic Modification: 285.5
Fusion: 5689.25
Protein Consequence: 2683.75
Gene Function: 1822.5
Rearrangements: 653.0
Copy Number: 207.0
Other: 673.5
Genotypes Easy: 195.0
Genotypes Compound: 117.5
Region Defined Variant: 8311.5
Transcript Variant: 346.5


In [51]:
not_supported_total_sum_impact_score =[v["total_sum_not_supported_category_impact"] for k, v in not_supported_variant_categories_impact_data.items()]
not_supported_total_number_evidence_items = [v["number_unique_not_supported_category_evidence"]for k, v in not_supported_variant_categories_evidence_summary_data.items()]
not_supported_percent_evidence_items = [v["percentage_accepted_evidence_not_supported_category_variants"]for k, v in not_supported_variant_categories_evidence_summary_data.items()]
not_supported_total_number_variants = [v["number_unique_not_supported_category_variants"]for k, v in not_supported_variant_categories_summary_data.items()]

In [52]:
not_supported_impact_dict = {
    "Category": VARIANT_CATEGORY_VALUES,
    "Total Sum Impact Score": not_supported_total_sum_impact_score,
    "Total Number Evidence Items": not_supported_total_number_evidence_items,
    "% Accepted Evidence Items": not_supported_percent_evidence_items,
    "Total Number Variants":not_supported_total_number_variants
}

In [53]:
not_supported_variant_impact_df = pd.DataFrame(not_supported_impact_dict)

# Summary

## Variant Analysis

In [54]:
all_variant_df = pd.DataFrame(VARIANT_ANALYSIS_SUMMARY)

In [55]:
all_variant_df["Percentage of all CIViC Variant Items"] = all_variant_df["Fraction of all CIViC Variant Items"].astype(str) + "  (" + all_variant_df["Percentage of all CIViC Variant Items"]+ ")"
all_variant_df["Percentage of Accepted Variant Items"] = all_variant_df["Fraction of Accepted Variant Items"].astype(str) + "  (" + all_variant_df["Percentage of Accepted Variant Items"]+ ")"
all_variant_df["Percentage of Not Accepted Variant Items"] = all_variant_df["Fraction of Not Accepted Variant Items"].astype(str) + "  (" + all_variant_df["Percentage of Not Accepted Variant Items"]+ ")"

In [56]:
all_variant_df = all_variant_df.drop(["Fraction of all CIViC Variant Items", "Fraction of Accepted Variant Items", "Fraction of Not Accepted Variant Items"], axis=1)

In [57]:
all_variant_percent_of_civic_df = all_variant_df.drop(["Percentage of Accepted Variant Items", "Percentage of Not Accepted Variant Items"], axis=1)
all_variant_percent_evidence_df = all_variant_df.drop(["Percentage of all CIViC Variant Items"], axis=1)

Summary Table 1: The table below shows the 3 categories that CIViC variants were divided into after normalization and what percentage they make up of all variants in CIViC data. 

In [58]:
all_variant_percent_of_civic_df = all_variant_percent_of_civic_df.set_index("Variant Category")
all_variant_percent_of_civic_df

,Percentage of all CIViC Variant Items
Variant Category,
Normalized,1869 / 3554 (52.59%)
Unable to Normalize,63 / 3554 (1.77%)
Not Supported,1622 / 3554 (45.64%)


In [59]:
summary_table_1 = all_variant_percent_of_civic_df

Summary Table 2: The table below shows the 3 categories that CIViC variants were divided into after normalization and what percentage of the variants in each category are accepted (have at least one evidence item that is accepted) or not.

In [60]:
all_variant_percent_evidence_df = all_variant_percent_evidence_df.set_index("Variant Category")
all_variant_percent_evidence_df

,Percentage of Accepted Variant Items,Percentage of Not Accepted Variant Items
Variant Category,,
Normalized,866 / 1869 (46.33%),1003 / 1869 (53.67%)
Unable to Normalize,9 / 63 (14.29%),54 / 63 (85.71%)
Not Supported,788 / 1622 (48.58%),834 / 1622 (51.42%)


In [61]:
summary_table_2 = all_variant_percent_evidence_df

In [62]:
not_supported_variant_categories = ["Expression","Epigenetic Modification", "Fusion", "Protein Consequence", "Gene Function", "Rearrangements", "Copy Number", "Other", "Genotypes Easy", "Genotypes Compound", "Region Defined Variant","Transcript Variant" ]
not_supported_fraction_of_civic_variant_items =[v["fraction_not_supported_category_variant_of_civic"] for v in not_supported_variant_categories_summary_data.values()]
not_supported_percent_of_civic_variant_items = [v["percent_not_supported_category_variant_of_civic"] for v in not_supported_variant_categories_summary_data.values()]
not_supported_fraction_not_supported_variant_items =[v["fraction_not_supported_category_variant_of_total_not_supported"] for v in not_supported_variant_categories_summary_data.values()]
not_supported_percent_not_supported_variant_items = [v["percent_not_supported_category_variant_of_total_not_supported"] for v in not_supported_variant_categories_summary_data.values()]
not_supported_fraction_of_accepted_variant_items =[v["fraction_accepted_not_supported_category_variants"] for v in not_supported_variant_categories_summary_data.values()]
not_supported_percent_of_accepted_variant_items = [v["percentage_accepted_not_supported_category_variants"] for v in not_supported_variant_categories_summary_data.values()]
not_supported_fraction_of_not_accepted_variant_items =[v["fraction_not_accepted_not_supported_category_variants"] for v in not_supported_variant_categories_summary_data.values()]
not_supported_percent_of_not_accepted_variant_items = [v["percentage_not_accepted_not_supported_category_variants"] for v in not_supported_variant_categories_summary_data.values()]

In [63]:
not_supported_variant_dict = {"Category": not_supported_variant_categories,
                                "Fraction of Not Supported Variant Items":not_supported_fraction_not_supported_variant_items ,
                                "Percent of Not Supported Variant Items":not_supported_percent_not_supported_variant_items,
                                "Fraction of all CIViC Variant Items": not_supported_fraction_of_civic_variant_items,
                                "Percent of all CIViC Variant Items":not_supported_percent_of_civic_variant_items,
                                "Fraction of Accepted Variant Items": not_supported_fraction_of_accepted_variant_items,
                                "Percent of Accepted Variant Items":not_supported_percent_of_accepted_variant_items,
                                "Fraction of Not Accepted Variant Items": not_supported_fraction_of_not_accepted_variant_items,
                                "Percent of Not Accepted Variant Items":not_supported_percent_of_not_accepted_variant_items}

In [64]:
not_supported_variant_df = pd.DataFrame(not_supported_variant_dict)

In [65]:
not_supported_variant_df["Percent of Not Supported Variant Items"] = not_supported_variant_df["Fraction of Not Supported Variant Items"].astype(str) + "  (" +  not_supported_variant_df["Percent of Not Supported Variant Items"]+ ")"
not_supported_variant_df["Percent of all CIViC Variant Items"] = not_supported_variant_df["Fraction of all CIViC Variant Items"].astype(str) + "  (" +  not_supported_variant_df["Percent of all CIViC Variant Items"]+ ")"
not_supported_variant_df["Percent of Accepted Variant Items"] = not_supported_variant_df["Fraction of Accepted Variant Items"].astype(str) + "  (" +  not_supported_variant_df["Percent of Accepted Variant Items"]+ ")"
not_supported_variant_df["Percent of Not Accepted Variant Items"] = not_supported_variant_df["Fraction of Not Accepted Variant Items"].astype(str) + "  (" +  not_supported_variant_df["Percent of Not Accepted Variant Items"]+ ")"

In [66]:
not_supported_variant_df = not_supported_variant_df.drop(["Fraction of Not Supported Variant Items","Fraction of all CIViC Variant Items","Fraction of Accepted Variant Items","Fraction of Not Accepted Variant Items"], axis=1)
not_supported_variant_df

,Category,Percent of Not Supported Variant Items,Percent of all CIViC Variant Items,Percent of Accepted Variant Items,Percent of Not Accepted Variant Items
0,Expression,291 / 1622 (17.94%),291 / 3554 (8.19%),180 / 291 (61.86%),111 / 291 (38.14%)
1,Epigenetic Modification,15 / 1622 (0.92%),15 / 3554 (0.42%),14 / 15 (93.33%),1 / 15 (6.67%)
2,Fusion,308 / 1622 (18.99%),308 / 3554 (8.67%),204 / 308 (66.23%),104 / 308 (33.77%)
3,Protein Consequence,128 / 1622 (7.89%),128 / 3554 (3.60%),70 / 128 (54.69%),58 / 128 (45.31%)
4,Gene Function,90 / 1622 (5.55%),90 / 3554 (2.53%),49 / 90 (54.44%),41 / 90 (45.56%)
5,Rearrangements,52 / 1622 (3.21%),52 / 3554 (1.46%),12 / 52 (23.08%),40 / 52 (76.92%)
6,Copy Number,34 / 1622 (2.10%),34 / 3554 (0.96%),19 / 34 (55.88%),15 / 34 (44.12%)
7,Other,93 / 1622 (5.73%),93 / 3554 (2.62%),43 / 93 (46.24%),50 / 93 (53.76%)
8,Genotypes Easy,11 / 1622 (0.68%),11 / 3554 (0.31%),8 / 11 (72.73%),3 / 11 (27.27%)
9,Genotypes Compound,6 / 1622 (0.37%),6 / 3554 (0.17%),4 / 6 (66.67%),2 / 6 (33.33%)


In [67]:
not_supported_variant_percent_of_civic_df =not_supported_variant_df.drop(["Percent of Not Supported Variant Items","Percent of Accepted Variant Items","Percent of Not Accepted Variant Items"], axis=1)
not_supported_variant_percent_of_not_supported_df = not_supported_variant_df.drop(["Percent of all CIViC Variant Items","Percent of Accepted Variant Items","Percent of Not Accepted Variant Items"], axis=1)
not_supported_variant_percent_evidence_df = not_supported_variant_df.drop(["Percent of all CIViC Variant Items","Percent of Not Supported Variant Items"], axis=1)

Summary Table 3: The table below shows the categories that the Not Supported variants were broken into and what percentage of all CIViC variants they make up. These percentages will not add up to 100% because Not Supported variants make up 45.62% of all CIViC variants. 

In [68]:
not_supported_variant_percent_of_civic_df = not_supported_variant_percent_of_civic_df.set_index("Category")
not_supported_variant_percent_of_civic_df

,Percent of all CIViC Variant Items
Category,
Expression,291 / 3554 (8.19%)
Epigenetic Modification,15 / 3554 (0.42%)
Fusion,308 / 3554 (8.67%)
Protein Consequence,128 / 3554 (3.60%)
Gene Function,90 / 3554 (2.53%)
Rearrangements,52 / 3554 (1.46%)
Copy Number,34 / 3554 (0.96%)
Other,93 / 3554 (2.62%)
Genotypes Easy,11 / 3554 (0.31%)


In [69]:
summary_table_3 = not_supported_variant_percent_of_civic_df

Summary Table 4: The table below shows the Not Supported variants broken up into 12 sub categories and what percent each sub category take up in Not Supported variant group.

In [70]:
not_supported_variant_percent_of_not_supported_df = not_supported_variant_percent_of_not_supported_df.set_index("Category")
not_supported_variant_percent_of_not_supported_df

,Percent of Not Supported Variant Items
Category,
Expression,291 / 1622 (17.94%)
Epigenetic Modification,15 / 1622 (0.92%)
Fusion,308 / 1622 (18.99%)
Protein Consequence,128 / 1622 (7.89%)
Gene Function,90 / 1622 (5.55%)
Rearrangements,52 / 1622 (3.21%)
Copy Number,34 / 1622 (2.10%)
Other,93 / 1622 (5.73%)
Genotypes Easy,11 / 1622 (0.68%)


In [71]:
summary_table_4: not_supported_variant_percent_of_not_supported_df

Summary Table 5: The table below shows the percent of variant items in each Not Supported variant sub category that are accepted(have at least one evidence item that is accepted) or not, per category.

In [72]:
not_supported_variant_percent_evidence_df = not_supported_variant_percent_evidence_df.set_index("Category")
not_supported_variant_percent_evidence_df

,Percent of Accepted Variant Items,Percent of Not Accepted Variant Items
Category,,
Expression,180 / 291 (61.86%),111 / 291 (38.14%)
Epigenetic Modification,14 / 15 (93.33%),1 / 15 (6.67%)
Fusion,204 / 308 (66.23%),104 / 308 (33.77%)
Protein Consequence,70 / 128 (54.69%),58 / 128 (45.31%)
Gene Function,49 / 90 (54.44%),41 / 90 (45.56%)
Rearrangements,12 / 52 (23.08%),40 / 52 (76.92%)
Copy Number,19 / 34 (55.88%),15 / 34 (44.12%)
Other,43 / 93 (46.24%),50 / 93 (53.76%)
Genotypes Easy,8 / 11 (72.73%),3 / 11 (27.27%)


In [73]:
summary_table_5= not_supported_variant_percent_evidence_df

## Evidence Analysis

In [74]:
all_variant_evidence_df = pd.DataFrame(EVIDENCE_ANALYSIS_SUMMARY)

In [75]:
all_variant_evidence_df["Percentage of all CIViC Evidence Items"] = all_variant_evidence_df["Fraction of all CIViC Evidence Items"].astype(str) + "  (" + all_variant_evidence_df["Percentage of all CIViC Evidence Items"]+ ")"
all_variant_evidence_df["Percentage of Accepted Evidence Items"] = all_variant_evidence_df["Fraction of Accepted Evidence Items"].astype(str) + "  (" + all_variant_evidence_df["Percentage of Accepted Evidence Items"]+ ")"
all_variant_evidence_df["Percentage of Submitted Evidence Items"] = all_variant_evidence_df["Fraction of Submitted Evidence Items"].astype(str) + "  (" + all_variant_evidence_df["Percentage of Submitted Evidence Items"]+ ")"

In [76]:
all_variant_evidence_df = all_variant_evidence_df.drop(["Fraction of all CIViC Evidence Items", "Fraction of Accepted Evidence Items", "Fraction of Submitted Evidence Items"], axis=1)

In [77]:
all_variant_evidence_percent_of_civic_df = all_variant_evidence_df.drop(["Percentage of Accepted Evidence Items", "Percentage of Submitted Evidence Items"], axis=1)	
all_variant_evidence_percent_evidence_df = all_variant_evidence_df.drop(["Percentage of all CIViC Evidence Items"], axis=1)

Summary Table 6: The table below shows what percentage of all evidence items in CIViC are associated with Normalized, Unable to Normalize, and Not Supported variants. This will not add up to 100% because evidence itmes may be used across multiple variants.

In [78]:
all_variant_evidence_percent_of_civic_df = all_variant_evidence_percent_of_civic_df.set_index("Variant Category")
all_variant_evidence_percent_of_civic_df

,Percentage of all CIViC Evidence Items
Variant Category,
Normalized,5917 / 9772 (60.55%)
Unable to Normalize,80 / 9772 (0.82%)
Not Supported,4293 / 9772 (43.93%)


In [79]:
summary_table_6 = all_variant_evidence_percent_of_civic_df

Summmary Table 7: The table below shows the percentage of accepted and sumbitted evidence items per category of variants. 

In [80]:
all_variant_evidence_percent_evidence_df = all_variant_evidence_percent_evidence_df.set_index("Variant Category")
all_variant_evidence_percent_evidence_df

,Percentage of Accepted Evidence Items,Percentage of Submitted Evidence Items
Variant Category,,
Normalized,2033 / 5917 (34.36%),3884 / 5917 (65.64%)
Unable to Normalize,14 / 80 (17.50%),66 / 80 (82.50%)
Not Supported,2201 / 4293 (51.27%),2092 / 4293 (48.73%)


In [81]:
summary_table_7 = all_variant_evidence_percent_evidence_df

In [82]:
not_supported_variant_categories = VARIANT_CATEGORY_VALUES
not_supported_fraction_of_civic_evidence_items = [v["fraction_not_supported_category_variant_evidence_of_civic"] for v in not_supported_variant_categories_evidence_summary_data.values()]
not_supported_percent_of_civic_evidence_items = [v["percent_not_supported_category_variant_evidence_of_civic"] for v in not_supported_variant_categories_evidence_summary_data.values()]
not_supported_fraction_not_supported_variant_evidence_items = [v["fraction_not_supported_category_variant_evidence_of_total_not_supported"] for v in not_supported_variant_categories_evidence_summary_data.values()]
not_supported_percent_not_supported_variant_evidence_items = [v["percent_not_supported_category_variant_evidence_of_total_not_supported"] for v in not_supported_variant_categories_evidence_summary_data.values()]
not_supported_fraction_of_accepted_evidence_items = [v["fraction_accepted_evidence_not_supported_category_variants"] for v in not_supported_variant_categories_evidence_summary_data.values()]
not_supported_percent_of_accepted_evidence_items = [v["percentage_accepted_evidence_not_supported_category_variants"] for v in not_supported_variant_categories_evidence_summary_data.values()]
not_supported_fraction_of_submitted_evidence_items = [v["fraction_submitted_evidence_not_supported_category_variants"]for v in not_supported_variant_categories_evidence_summary_data.values()]
not_supported_percent_of_submitted_evidence_items = [v["percentage_submitted_evidence_not_supported_category_variants"] for v in not_supported_variant_categories_evidence_summary_data.values()]

In [83]:
not_supported_variant_evidence_dict = {"Category": not_supported_variant_categories,
                                "Fraction of Not Supported Variant Evidence Items": not_supported_fraction_not_supported_variant_evidence_items,
                                "Percent of Not Supported Variant Evidence Items": not_supported_percent_not_supported_variant_evidence_items,
                                "Fraction of CIViC Evidence Items": not_supported_fraction_of_civic_evidence_items,
                                "Percent of CIViC Evidence Items": not_supported_percent_of_civic_evidence_items,
                                "Fraction of Accepted Evidence Items": not_supported_fraction_of_accepted_evidence_items,
                                "Percent of Accepted Evidence Items": not_supported_percent_of_accepted_evidence_items,
                                "Fraction of Submitted Evidence Items": not_supported_fraction_of_submitted_evidence_items,
                                "Percent of Submitted Evidence Items":not_supported_percent_of_submitted_evidence_items}

In [84]:
not_supported_variant_evidence_df = pd.DataFrame(not_supported_variant_evidence_dict)

In [85]:
not_supported_variant_evidence_df["Percent of CIViC Evidence Items"] = not_supported_variant_evidence_df["Fraction of CIViC Evidence Items"].astype(str) + "  (" + not_supported_variant_evidence_df["Percent of CIViC Evidence Items"]+ ")"
not_supported_variant_evidence_df["Percent of Not Supported Variant Evidence Items"] = not_supported_variant_evidence_df["Fraction of Not Supported Variant Evidence Items"].astype(str) + "  ("+ not_supported_variant_evidence_df["Percent of Not Supported Variant Evidence Items"]+ ")"
not_supported_variant_evidence_df["Percent of Accepted Evidence Items"] = not_supported_variant_evidence_df["Fraction of Accepted Evidence Items"].astype(str) + "  ("+ not_supported_variant_evidence_df["Percent of Accepted Evidence Items"]+ ")"
not_supported_variant_evidence_df["Percent of Submitted Evidence Items"] = not_supported_variant_evidence_df["Fraction of Submitted Evidence Items"].astype(str) + "  ("+ not_supported_variant_evidence_df["Percent of Submitted Evidence Items"]+ ")"

In [86]:
not_supported_variant_evidence_df = not_supported_variant_evidence_df.drop(["Fraction of CIViC Evidence Items", "Fraction of Not Supported Variant Evidence Items", "Fraction of Accepted Evidence Items", "Fraction of Submitted Evidence Items"], axis=1)

In [87]:
not_supported_variant_evidence_percent_of_civic_df = not_supported_variant_evidence_df.drop(["Percent of Not Supported Variant Evidence Items", "Percent of Accepted Evidence Items", "Percent of Submitted Evidence Items"], axis=1) 
not_supported_variant_evidence_percent_of_not_supported_df = not_supported_variant_evidence_df.drop(["Percent of CIViC Evidence Items",  "Percent of Accepted Evidence Items", "Percent of Submitted Evidence Items"], axis=1) 
not_supported_variant_evidence_percent_evidence_df = not_supported_variant_evidence_df.drop(["Percent of Not Supported Variant Evidence Items", "Percent of CIViC Evidence Items"], axis=1) 

Summary Table 8: The table below shows the percentage of all CIViC evidenec items that are associated with a Not Supported variant sub category. This will not add up to 100% since the evidence items can be associated with multiple variants.

In [88]:
not_supported_variant_evidence_percent_of_civic_df = not_supported_variant_evidence_percent_of_civic_df.set_index("Category")
not_supported_variant_evidence_percent_of_civic_df

,Percent of CIViC Evidence Items
Category,
Expression,614 / 9772 (6.28%)
Epigenetic Modification,24 / 9772 (0.25%)
Fusion,1221 / 9772 (12.49%)
Protein Consequence,301 / 9772 (3.08%)
Gene Function,347 / 9772 (3.55%)
Rearrangements,137 / 9772 (1.40%)
Copy Number,64 / 9772 (0.65%)
Other,162 / 9772 (1.66%)
Genotypes Easy,23 / 9772 (0.24%)


In [89]:
summary_table_8 = not_supported_variant_evidence_percent_of_civic_df

Summary Table 9: The table below shows the percentage of all evidenec items associated with Not Supported variants that are associated with a variant sub category. 

In [90]:
not_supported_variant_evidence_percent_of_not_supported_df = not_supported_variant_evidence_percent_of_not_supported_df.set_index('Category')
not_supported_variant_evidence_percent_of_not_supported_df

,Percent of Not Supported Variant Evidence Items
Category,
Expression,614 / 4293 (14.30%)
Epigenetic Modification,24 / 4293 (0.56%)
Fusion,1221 / 4293 (28.44%)
Protein Consequence,301 / 4293 (7.01%)
Gene Function,347 / 4293 (8.08%)
Rearrangements,137 / 4293 (3.19%)
Copy Number,64 / 4293 (1.49%)
Other,162 / 4293 (3.77%)
Genotypes Easy,23 / 4293 (0.54%)


In [91]:
summary_table_9 = not_supported_variant_evidence_percent_of_not_supported_df

Summary Table 10: The table below shows the percentage of evidence items associated with Not Supported variant sub categories that are accepted or submitted.

In [92]:
not_supported_variant_evidence_percent_evidence_df= not_supported_variant_evidence_percent_evidence_df.set_index("Category")
not_supported_variant_evidence_percent_evidence_df

,Percent of Accepted Evidence Items,Percent of Submitted Evidence Items
Category,,
Expression,342 / 614 (55.70%),272 / 614 (44.30%)
Epigenetic Modification,22 / 24 (91.67%),2 / 24 (8.33%)
Fusion,724 / 1221 (59.30%),497 / 1221 (40.70%)
Protein Consequence,191 / 301 (63.46%),110 / 301 (36.54%)
Gene Function,154 / 347 (44.38%),193 / 347 (55.62%)
Rearrangements,53 / 137 (38.69%),84 / 137 (61.31%)
Copy Number,27 / 64 (42.19%),37 / 64 (57.81%)
Other,60 / 162 (37.04%),102 / 162 (62.96%)
Genotypes Easy,13 / 23 (56.52%),10 / 23 (43.48%)


In [93]:
summary_table_10 = not_supported_variant_evidence_percent_evidence_df

## Impact

The bar graph below shows the relationship between the Not Suported variant sub category impact score and the sub category. Additionally, the colors illustrate the number of evidence items associated each sub category.

In [94]:
fig = px.bar(not_supported_variant_impact_df, x="Category", 
             y="Total Sum Impact Score",
             hover_data=["Total Number Evidence Items", 
             not_supported_variant_impact_df["% Accepted Evidence Items"]], 
             color="Total Number Evidence Items",
             labels={"Total Sum Impact Score":"Total Sum Impact Score"},
             text_auto= ".1f",
             color_continuous_scale= "geyser")
fig.update_traces(width=1)
fig.show()

In [95]:
#fig.write_html("../../../../../../civic_ns_categories_impact_redgreen.html")

The scatterplot below shows the relationship between the Not Suported variant sub category impact score and the number of evidence items associated with variants in each sub category. Additionally, the sizes of the data point represent the number of variants in each sub category. 

In [96]:
fig2= px.scatter(data_frame= not_supported_variant_impact_df,
                x= "Total Number Evidence Items",
                y= "Total Sum Impact Score",
                size= "Total Number Variants",
                size_max= 40,
                text= "Total Number Variants",
                # color_discrete_sequence= Bold,
                color= "Category",
                hover_data= "% Accepted Evidence Items")
fig2.show()

In [97]:
#fig2.write_html("../../../../../../civic_ns_categories_impact_scatterplot.html")